In [ ]:
pip install numpy

# Questão 1


In [ ]:
import numpy as np
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

# Configuração
VETOR_TAMANHO = 50000
SEED = 42
NUM_PROCURADO = 12345  # Escolha um número presente ou ausente
NUM_WORKERS = 4  # Pode ajustar de acordo com seu CPU

# Gerar vetor com números aleatórios
np.random.seed(SEED)
vetor = np.random.randint(0, 20000, VETOR_TAMANHO)  # Número de até 20.000

# Serial
def busca_serial(vetor, alvo):
    for i, v in enumerate(vetor):
        if v == alvo:
            return i
    return None

# Paralela
def busca_parcial(parte, offset, alvo):
    for i, v in enumerate(parte):
        if v == alvo:
            return offset + i
    return None

def busca_paralela(vetor, alvo, num_workers=4):
    tamanho_pedaco = len(vetor) // num_workers
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futuros = []
        for i in range(num_workers):
            inicio = i * tamanho_pedaco
            fim = (i + 1) * tamanho_pedaco if i != num_workers - 1 else len(vetor)
            futuros.append(executor.submit(busca_parcial, vetor[inicio:fim], inicio, alvo))

        for futuro in as_completed(futuros):
            resultado = futuro.result()
            if resultado is not None:
                # Cancela os outros se já encontrou
                for f in futuros:
                    f.cancel()
                return resultado
    return None

# Execução e comparação de tempo
if __name__ == "__main__":
    # Serial
    t1 = time.time()
    pos_serial = busca_serial(vetor, NUM_PROCURADO)
    t2 = time.time()
    tempo_serial = t2 - t1

    # Paralela
    t3 = time.time()
    pos_paralela = busca_paralela(vetor, NUM_PROCURADO, NUM_WORKERS)
    t4 = time.time()
    tempo_paralela = t4 - t3

    speedup = tempo_serial / tempo_paralela if tempo_paralela > 0 else float('inf')

    print(f"Serial: posição = {pos_serial}, tempo = {tempo_serial:.4f} s")
    print(f"Paralela: posição = {pos_paralela}, tempo = {tempo_paralela:.4f} s")
    print(f"Speedup: {speedup:.2f}x")


# Questão 2

In [ ]:
import random
import time
import multiprocessing
import numpy as np
from concurrent.futures import ProcessPoolExecutor

# Tamanho do vetor
SIZE = 5000000

def create_random_array(size=SIZE):
    """Cria um vetor de inteiros aleatórios."""
    return [random.randint(0, 100000) for _ in range(size)]

# ---------- IMPLEMENTAÇÃO SERIAL ----------

def serial_sort(arr):
    """Ordenação serial usando o método nativo do Python (TimSort)."""
    return sorted(arr)

# ---------- IMPLEMENTAÇÃO PARALELA ----------

def merge(left, right):
    """Mescla dois arrays ordenados."""
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

def merge_sort_chunk(arr):
    """Ordena uma parte do array."""
    return sorted(arr)

def parallel_sort(arr, num_processes=None):
    """Ordenação paralela usando multiprocessamento."""
    if num_processes is None:
        num_processes = multiprocessing.cpu_count()

    # Determina o tamanho de cada parte
    chunk_size = len(arr) // num_processes

    # Divide o array em partes
    chunks = [arr[i:i+chunk_size] for i in range(0, len(arr), chunk_size)]

    # Ordena cada parte em paralelo
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        sorted_chunks = list(executor.map(merge_sort_chunk, chunks))

    # Mescla as partes ordenadas
    result = sorted_chunks[0]
    for chunk in sorted_chunks[1:]:
        result = merge(result, chunk)

    return result

# ---------- IMPLEMENTAÇÃO PARALELA OTIMIZADA ----------

# Funções globais para a versão otimizada
def sort_partition(args):
    """Ordena uma partição do array."""
    arr, idx = args
    partition = arr[idx]
    return np.sort(partition)

def merge_arrays(arrays):
    """Mescla arrays ordenados."""
    if len(arrays) == 1:
        return arrays[0]
    mid = len(arrays) // 2
    left = merge_arrays(arrays[:mid])
    right = merge_arrays(arrays[mid:])
    # Combina e ordena
    merged = np.concatenate([left, right])
    merged.sort()
    return merged

def optimized_parallel_sort(arr):
    """Versão otimizada da ordenação paralela usando numpy e multiprocessamento."""
    # Converte para array numpy
    np_arr = np.array(arr)

    # Divide o trabalho entre os cores disponíveis
    num_cores = 3
    split_indices = np.array_split(np.arange(len(np_arr)), num_cores)

    # Prepara os argumentos
    args = [(np_arr, idx) for idx in split_indices]

    # Ordena as partições em paralelo
    with ProcessPoolExecutor(max_workers=num_cores) as executor:
        sorted_partitions = list(executor.map(sort_partition, args))

    # Retorna o resultado como uma lista Python
    return merge_arrays(sorted_partitions).tolist()

def run_optimized_benchmark():
    """Executa os testes com a versão otimizada."""
    # Cria o mesmo vetor aleatório para ambos os testes
    array = create_random_array()
    array_copy = array.copy()

    # Testa a versão serial
    start_time = time.time()
    sorted_serial = serial_sort(array)
    serial_time = time.time() - start_time
    print(f"Tempo de execução serial: {serial_time:.6f} segundos")

    # Testa a versão paralela otimizada
    start_time = time.time()
    sorted_parallel = optimized_parallel_sort(array_copy)
    parallel_time = time.time() - start_time
    print(f"Tempo de execução paralelo otimizado: {parallel_time:.6f} segundos")

    # Verifica se os resultados são idênticos
    is_correct = sorted_serial == sorted_parallel
    print(f"Os resultados são idênticos? {is_correct}")

    # Calcula o speedup
    speedup = serial_time / parallel_time
    print(f"Speedup: {speedup:.2f}x")

    # Verifica se o speedup atende ao requisito
    if speedup >= 1.7:
        print("✓ O requisito de speedup de 1.7x foi atingido.")
    else:
        print("✗ O requisito de speedup de 1.7x NÃO foi atingido.")

    return serial_time, parallel_time, speedup

if __name__ == "__main__":

    print("\n===== BENCHMARK OTIMIZADO =====")
    run_optimized_benchmark()



# Questão 3

In [43]:
import numpy as np
import time
from multiprocessing import Pool, cpu_count

def matrix_mult_serial(A, B):
    """Multiplicação de matrizes - versão serial"""
    rows_A = A.shape[0]
    cols_A = A.shape[1]
    cols_B = B.shape[1]

    # Inicializa a matriz resultado com zeros
    result = np.zeros((rows_A, cols_B))

    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]

    return result

def multiply_row(args):
    """Função auxiliar para processamento paralelo de uma linha"""
    row, A, B = args
    cols_B = B.shape[1]
    cols_A = A.shape[1]
    result_row = np.zeros(cols_B)

    for j in range(cols_B):
        for k in range(cols_A):
            result_row[j] += A[row][k] * B[k][j]

    return row, result_row

def matrix_mult_parallel(A, B):
    """Multiplicação de matrizes - versão paralela"""
    rows_A = A.shape[0]
    result = np.zeros((rows_A, B.shape[1]))

    # Criando lista de argumentos para cada linha
    args = [(i, A, B) for i in range(rows_A)]

    # Usando Pool para processamento paralelo
    with Pool(processes=9) as pool:
        results = pool.map(multiply_row, args)

    # Reorganizando os resultados
    for row, row_result in results:
        result[row] = row_result

    return result

def main():
    # Criando matrizes de exemplo
    print("Criando matrizes A (200x400) e B (400x100)...")
    A = np.random.rand(200, 400)
    B = np.random.rand(400, 100)

    # Multiplicação serial
    print("\nExecutando multiplicação serial...")
    start_time = time.time()
    result_serial = matrix_mult_serial(A, B)
    serial_time = time.time() - start_time
    print(f"Tempo de execução serial: {serial_time:.4f} segundos")

    # Multiplicação paralela
    print("\nExecutando multiplicação paralela...")
    start_time = time.time()
    result_parallel = matrix_mult_parallel(A, B)
    parallel_time = time.time() - start_time
    print(f"Tempo de execução paralelo: {parallel_time:.4f} segundos")
    print(f"Número de núcleos utilizados: {cpu_count()}")

    # Comparação e cálculo do speedup
    print("\nVerificando resultados...")
    assert np.allclose(result_serial, result_parallel), "Resultados diferentes!"

    speedup = serial_time / parallel_time

    print("\nComparação de desempenho:")
    print(f"Tempo serial: {serial_time:.4f} segundos")
    print(f"Tempo paralelo: {parallel_time:.4f} segundos")
    print(f"Speedup: {speedup:.2f}x")

    if speedup >= 1.5:
        print("\n✅ Speedup maior ou igual a 1.5x alcançado!")
    else:
        print("\n⚠️ Speedup menor que 1.5x. Considere:")
        print("- Usar matrizes maiores para melhorar a relação computação/comunicação")
        print("- Otimizar a implementação paralela")
        print("- Verificar se há outros processos consumindo recursos da CPU")

    return result_parallel

if __name__ == "__main__":
    result_matrix = main()

Criando matrizes A (200x400) e B (400x100)...

Executando multiplicação serial...
Tempo de execução serial: 9.2102 segundos

Executando multiplicação paralela...
Tempo de execução paralelo: 7.2572 segundos
Número de núcleos utilizados: 2

Verificando resultados...

Comparação de desempenho:
Tempo serial: 9.2102 segundos
Tempo paralelo: 7.2572 segundos
Speedup: 1.27x

⚠️ Speedup menor que 1.5x. Considere:
- Usar matrizes maiores para melhorar a relação computação/comunicação
- Otimizar a implementação paralela
- Verificar se há outros processos consumindo recursos da CPU


# Questão 4

In [41]:
import time
from concurrent.futures import ProcessPoolExecutor
import numpy as np

def is_prime(n):
    if n < 2 or n % 2 == 0 and n != 2:
        return False
    for i in range(3, int(n**.5)+1, 2):
        if n % i == 0:
            return False
    return True

def serial_primes(B):
    primes = []
    for num in B:
        if is_prime(num):
            primes.append(num)
    return primes

def worker(chunk):
    return [num for num in chunk if is_prime(num)]


if __name__ == "__main__":
    NUM_WORKERS = 8
    size = 1_000_000  # Aumentando para 100.000 elementos
    max_num = 1_000_000

    print(f"Gerando array com {size} números entre 1 e {max_num}...")
    np.random.seed(42)
    A = np.random.randint(1, max_num, size=size)
    #A = np.genfromtxt('vet_exerc_B.csv', delimiter=',').astype(int)
    chunks = np.array_split(A, NUM_WORKERS)

    start_time_serial = time.time()
    primes_serial = serial_primes(A)
    end_time_serial = time.time()

    print(f'Numeros primos encontrados:{len(primes_serial)}')

    print(f"Serial execution time: {end_time_serial - start_time_serial:.2f} seconds")


    start_time = time.time()
    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as pool:
        results = pool.map(worker, chunks)

    primes_parallel = [prime for chunk in results for prime in chunk]

    end_time = time.time()

    print(f'Numeros primos do paralelo encontrados: {len(primes_parallel)}')

    print(f"Parallel execution time: {(end_time_serial - start_time_serial)/(end_time-start_time):.2f} seconds")

Gerando array com 1000000 números entre 1 e 1000000...
Numeros primos encontrados:78397
Serial execution time: 6.88 seconds
Numeros primos do paralelo encontrados: 78397
Parallel execution time: 0.82 seconds
